# Notebook for CME Futures Challenge

### The Rough Idea

Model indices as geometric brownian motion (dS/S = mudt + sigmadB)  
Model mu (market line) as a linear regression with numerous factors including economic, credit measures, etc  
Model sigma as a function of volatility including recent volatility and EMA (decay)  
Long/short based on futures mispricings based on our model  

# Downloading historical data for indices (S&P, NASDAQ, DJIA)

Imports

In [44]:
import yfinance as yf
import pandas as pd
import plotly.express as px
from typing import List, Dict

Make get_data function for downloading from yf

In [45]:
def get_data(tickers: List):
    data_dictionary = {}
    for ticker in tickers:
        data_dictionary[ticker] = yf.download(ticker, period='360mo', interval='1d')
    return data_dictionary

Now let's get data for indices and display with pd

In [46]:
indices = ['^GSPC', '^IXIC', '^DJI'] # S&P, NASDAQ, DJIA
data_dictionary = get_data(indices)

s_p = pd.DataFrame(data_dictionary['^GSPC'])
nasdaq = pd.DataFrame(data_dictionary['^IXIC'])
djia = pd.DataFrame(data_dictionary['^DJI'])

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\16652999.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\16652999.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\16652999.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [47]:
s_p

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
1995-09-20,586.770020,586.770020,584.179993,584.200012,400050000
1995-09-21,583.000000,586.789978,580.909973,586.770020,367100000
1995-09-22,581.729980,583.000000,578.250000,583.000000,370790000
1995-09-25,581.809998,582.140015,579.500000,581.729980,273120000
1995-09-26,581.409973,584.659973,580.650024,581.809998,363630000
...,...,...,...,...,...
2025-09-15,6615.279785,6619.620117,6602.069824,6603.490234,5045020000
2025-09-16,6606.759766,6626.990234,6600.109863,6624.129883,5359510000


We need to flatten this - notice ticker header

In [48]:
s_p = s_p.droplevel(1, axis=1)
nasdaq = nasdaq.droplevel(1, axis=1)
djia = djia.droplevel(1, axis=1)

In [49]:
s_p

Price,Close,High,Low,Open,Volume
Date,,,,,
1995-09-20,586.770020,586.770020,584.179993,584.200012,400050000
1995-09-21,583.000000,586.789978,580.909973,586.770020,367100000
1995-09-22,581.729980,583.000000,578.250000,583.000000,370790000
1995-09-25,581.809998,582.140015,579.500000,581.729980,273120000
1995-09-26,581.409973,584.659973,580.650024,581.809998,363630000
...,...,...,...,...,...
2025-09-15,6615.279785,6619.620117,6602.069824,6603.490234,5045020000
2025-09-16,6606.759766,6626.990234,6600.109863,6624.129883,5359510000
2025-09-17,6600.350098,6624.390137,6551.149902,6604.870117,5805340000


Let's drop high, low, and open and rename columns

In [50]:
s_p.drop(columns=['High', 'Low', 'Open'], inplace=True)
nasdaq.drop(columns=['High', 'Low', 'Open'], inplace=True)
djia.drop(columns=['High', 'Low', 'Open'], inplace=True)

s_p = s_p.rename(columns={'Close': 'S&P_Close', 'Volume': 'S&P_Volume'})
nasdaq = nasdaq.rename(columns={'Close': 'NASDAQ_Close', 'Volume': 'NASDAQ_Volume'})
djia = djia.rename(columns={'Close': 'DJIA_Close', 'Volume': 'DJIA_Volume'})

Let's get a quick plot of an index

In [51]:
fig = px.line(s_p, x=s_p.index, y="S&P_Close", title="S&P Daily Past 30 Years")
fig.show()

# Downloading historical data for our factor model

We are going to model the index as a geometric brownian motion, with the mu factor being a linear regression model with numerous inputs.  

## Factor considerations:  
### <u>Term structure</u>
###### Term spread (10Y-3M)

### <u>Credit conditions</u>
###### IG spread (BAA-AAA)

### <u>Valuation</u>
###### Forward E/P - real 10Y
###### Dividend yield

### <u>Economic</u>
###### Fed funds
###### Inflation (CPI)
###### DXY change (dollar index)  

### Some of these we can get from yahoo finance:  

In [52]:
tickers = [
    # Term structure
    '^TNX', # 10yr CBOE
    '^IRX', # 3m bill (on discount basis, need to convert to yield)

    # Economic
    'DX-Y.NYB', # Dollar index
]

data_dictionary = get_data(tickers)

ten_yr = pd.DataFrame(data_dictionary['^TNX']['Close'])
three_m = pd.DataFrame(data_dictionary['^IRX']['Close'])
dollar_index = pd.DataFrame(data_dictionary['DX-Y.NYB']['Close'])

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\16652999.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\16652999.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\16652999.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Rename columns

In [53]:
ten_yr = ten_yr.rename(columns={'^TNX': 'ten_yr'})
three_m = three_m.rename(columns={'^IRX': 'three_m'})
dollar_index = dollar_index.rename(columns={'DX-Y.NYB': 'dollar_index'})

We should get dividend yield too

In [54]:
etfs = ['SPY', 'QQQ', 'DIA']
div_data = {}

for etf in etfs:
    ticker = yf.Ticker(etf)
    div = ticker.dividends
    price = ticker.history('372mo')['Close']

    # Calculate dividend yield
    div_12m = div.rolling(window='365D', min_periods=1).sum()
    div_12m = div_12m.reindex(price.index, method='ffill')
    div_yield = div_12m / price
    div_data[etf] = div_yield

Fix index for all 3 and rename columns

In [55]:
div_data['SPY'].index = pd.to_datetime(div_data['SPY'].index).normalize().tz_localize(None) # Normalize puts date in format we want
div_data['QQQ'].index = pd.to_datetime(div_data['QQQ'].index).normalize().tz_localize(None) # Localize (none) makes sure it doesn't add our timezone
div_data['DIA'].index = pd.to_datetime(div_data['DIA'].index).normalize().tz_localize(None)

div_data['SPY'].name = 'SPY_div'
div_data['QQQ'].name = 'QQQ_div'
div_data['DIA'].name = 'DIA_div'

In [56]:
div_data['SPY']

Date
1994-09-20    0.054990
1994-09-21    0.054990
1994-09-22    0.055120
1994-09-23    0.055308
1994-09-26    0.055027
                ...   
2025-09-15    0.013546
2025-09-16    0.013565
2025-09-17    0.013582
2025-09-18    0.013518
2025-09-19    0.013560
Name: SPY_div, Length: 7803, dtype: float64

### pandas_datareader lets us download fred data

In [57]:
from pandas_datareader import data as pdr
from datetime import datetime

In [58]:
start = datetime(1990,1,1) # Start date for download

# Macroeconomic data
gdp = pdr.DataReader("GDP", "fred", start)
cpi = pdr.DataReader("CPIAUCSL", "fred", start)
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start)

# For some reason this download doesn't have the most recent fed funds rate
fedfunds = pd.concat([fedfunds['FEDFUNDS'], pd.Series([4.08], index=[datetime(2025,9,17)])])

# Credit risk data
ig_spread = pdr.DataReader("BAMLC0A4CBBB", "fred", start)   # BofA BBB corp minus Treasuries
#hy_spread = pdr.DataReader("BAMLH0A0HYM2", "fred", start)   # BofA US High Yield spread
#baa_spread = pdr.DataReader("BAA10Y", "fred", start)        # Moody’s Baa – 10Y Treasury

Rename series

In [59]:
cpi.name = 'CPI'
fedfunds.name = 'fed_funds'
ig_spread.name = 'credit_spread'

In [60]:
fred_data = [gdp, cpi, fedfunds, ig_spread]

# Last business day <= today
last_bday = pd.bdate_range(end=pd.Timestamp.today().normalize().tz_localize(None), periods=1)[0]

for i, df in enumerate(fred_data):
    s = df.squeeze() # make it a Series
    # Build a business-day index from the series start to last_bday
    bidx = pd.bdate_range(start=s.index.min(), end=last_bday)
    # Reindex to business days and forward-fill
    s = s.reindex(bidx, method='ffill')
    # Write back as a 1-col DataFrame with a proper name
    name = s.name if s.name else f"series_{i}"
    fred_data[i] = s.to_frame(name)

In [61]:
fred_data[0]

,GDP
1990-01-01,5872.701
1990-01-02,5872.701
1990-01-03,5872.701
1990-01-04,5872.701
1990-01-05,5872.701
...,...
2025-09-15,30353.902
2025-09-16,30353.902
2025-09-17,30353.902
2025-09-18,30353.902


Let's build a master dataframe

In [62]:
data = s_p.join([nasdaq, djia, div_data['SPY'], div_data['QQQ'], div_data['DIA'], ten_yr, three_m, dollar_index, fred_data[0], fred_data[1], fred_data[2], fred_data[3]])
data

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,three_m,dollar_index,GDP,CPIAUCSL,fed_funds,BAMLC0A4CBBB
Date,,,,,,,,,,,,,,,,
1995-09-20,586.770020,4.000500e+08,1065.089966,5.148200e+08,4792.689941,52430000.0,0.044446,NaN,NaN,6.095,5.170,85.959999,7683.125,153.100,5.80,NaN
1995-09-21,583.000000,3.671000e+08,1058.510010,4.696900e+08,4767.399902,38620000.0,0.044815,NaN,NaN,6.200,5.230,83.699997,7683.125,153.100,5.80,NaN
1995-09-22,581.729980,3.707900e+08,1053.390015,4.390400e+08,4764.149902,38200000.0,0.044803,NaN,NaN,6.233,5.200,84.269997,7683.125,153.100,5.80,NaN
1995-09-25,581.809998,2.731200e+08,1046.150024,3.126700e+08,4769.930176,34520000.0,0.044875,NaN,NaN,6.229,5.240,84.550003,7683.125,153.100,5.80,NaN
1995-09-26,581.409973,3.636300e+08,1038.050049,4.266700e+08,4765.600098,35490000.0,0.044887,NaN,NaN,6.259,5.280,84.699997,7683.125,153.100,5.80,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-15,6615.279785,5.045020e+09,22348.750000,8.904030e+09,45883.449219,401500000.0,0.013546,0.006052,0.015259,4.034,3.900,97.300003,30353.902,323.364,4.33,0.95
2025-09-16,6606.759766,5.359510e+09,22333.960938,8.592240e+09,45757.898438,443400000.0,0.013565,0.006057,0.015301,4.026,3.890,96.629997,30353.902,323.364,4.33,0.96
2025-09-17,6600.350098,5.805340e+09,22261.330078,9.325980e+09,46018.320312,509830000.0,0.013582,0.006069,0.015219,4.076,3.868,96.870003,30353.902,323.364,4.08,0.96


# Linear regression model

### Feature Engineering

We need to be careful to not include things such as raw moving averages that will leak volatility information into our drift prediction  

In [63]:
import numpy as np

Function definitions to help out

In [64]:
def rolling_mean(data, window):
    return data.rolling(window, min_periods=window).mean()

Features

#  (TODO: look at making features like diffs for economic metrics, figure out when economic metrics are released vs reported in data)

In [65]:
# First, make log prices / volumes of our data, then log normal assumptions are better and everything is additive
data['S&P_log_price'] = np.log(data['S&P_Close'])
data['NASDAQ_log_price'] = np.log(data['NASDAQ_Close'])
data['DJIA_log_price'] = np.log(data['DJIA_Close'])

data['S&P_log_volume'] = np.log(data['S&P_Volume'])
data['NASDAQ_log_volume'] = np.log(data['NASDAQ_Volume'])
data['DJIA_log_volume'] = np.log(data['DJIA_Volume'])

# Monthly log returns 
data['S&P_ret'] = data['S&P_log_price'].diff(21)
data['NASDAQ_ret'] = data['NASDAQ_log_price'].diff(21)
data['DJIA_ret'] = data['DJIA_log_price'].diff(21)

# Next month log returns -- This will be our target variable
data[['S&P_next_ret','NASDAQ_next_ret','DJIA_next_ret']] = data[['S&P_ret','NASDAQ_ret','DJIA_ret']].shift(-1).dropna()


# ===== S&P =====
# Price-based
data['S&P_mom_1w'] = data['S&P_log_price'].diff(5) # Total price change / momentum indicator
data['S&P_mom_3m'] = data['S&P_log_price'].diff(63)
data['S&P_3m_rolling_price'] = rolling_mean(data['S&P_log_price'], 63)
data['S&P_trend_speed_price'] = data['S&P_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['S&P_trend_dist_price'] = data['S&P_log_price'] - data['S&P_3m_rolling_price']


# Volume-based (essentially the same as price for now)
data['S&P_vlm_1w'] = data['S&P_log_volume'].diff(5) # Total volume change / momentum indicator
data['S&P_vlm_1m'] = data['S&P_log_volume'].diff(21)
data['S&P_vlm_3m'] = data['S&P_log_volume'].diff(63)
data['S&P_3m_rolling_volume'] = rolling_mean(data['S&P_log_volume'], 63)
data['S&P_trend_speed_volume'] = data['S&P_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['S&P_trend_dist_volume'] = data['S&P_log_volume'] - data['S&P_3m_rolling_volume']

# ===== NASDAQ =====
# Price-based
data['NASDAQ_mom_1w'] = data['NASDAQ_log_price'].diff(5)  # Total price change / momentum indicator
data['NASDAQ_mom_3m'] = data['NASDAQ_log_price'].diff(63)
data['NASDAQ_3m_rolling_price'] = rolling_mean(data['NASDAQ_log_price'], 63)
data['NASDAQ_trend_speed_price'] = data['NASDAQ_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['NASDAQ_trend_dist_price'] = data['NASDAQ_log_price'] - data['NASDAQ_3m_rolling_price']

# Volume-based (essentially the same as price for now)
data['NASDAQ_vlm_1w'] = data['NASDAQ_log_volume'].diff(5)  # Total volume change / momentum indicator
data['NASDAQ_vlm_1m'] = data['NASDAQ_log_volume'].diff(21)
data['NASDAQ_vlm_3m'] = data['NASDAQ_log_volume'].diff(63)
data['NASDAQ_3m_rolling_volume'] = rolling_mean(data['NASDAQ_log_volume'], 63)
data['NASDAQ_trend_speed_volume'] = data['NASDAQ_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['NASDAQ_trend_dist_volume'] = data['NASDAQ_log_volume'] - data['NASDAQ_3m_rolling_volume']

# ===== DJIA =====
# Price-based
data['DJIA_mom_1w'] = data['DJIA_log_price'].diff(5)  # Total price change / momentum indicator
data['DJIA_mom_3m'] = data['DJIA_log_price'].diff(63)
data['DJIA_3m_rolling_price'] = rolling_mean(data['DJIA_log_price'], 63)
data['DJIA_trend_speed_price'] = data['DJIA_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['DJIA_trend_dist_price'] = data['DJIA_log_price'] - data['DJIA_3m_rolling_price']

# Volume-based (essentially the same as price for now)
data['DJIA_vlm_1w'] = data['DJIA_log_volume'].diff(5)  # Total volume change / momentum indicator
data['DJIA_vlm_1m'] = data['DJIA_log_volume'].diff(21)
data['DJIA_vlm_3m'] = data['DJIA_log_volume'].diff(63)
data['DJIA_3m_rolling_volume'] = rolling_mean(data['DJIA_log_volume'], 63)
data['DJIA_trend_speed_volume'] = data['DJIA_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['DJIA_trend_dist_volume'] = data['DJIA_log_volume'] - data['DJIA_3m_rolling_volume']






C:\Users\rjsyo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

divide by zero encountered in log



In [66]:
data

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,DJIA_mom_3m,DJIA_3m_rolling_price,DJIA_trend_speed_price,DJIA_trend_dist_price,DJIA_vlm_1w,DJIA_vlm_1m,DJIA_vlm_3m,DJIA_3m_rolling_volume,DJIA_trend_speed_volume,DJIA_trend_dist_volume
Date,,,,,,,,,,,,,,,,,,,,,
1995-09-20,586.770020,4.000500e+08,1065.089966,5.148200e+08,4792.689941,52430000.0,0.044446,NaN,NaN,6.095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-21,583.000000,3.671000e+08,1058.510010,4.696900e+08,4767.399902,38620000.0,0.044815,NaN,NaN,6.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-22,581.729980,3.707900e+08,1053.390015,4.390400e+08,4764.149902,38200000.0,0.044803,NaN,NaN,6.233,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-25,581.809998,2.731200e+08,1046.150024,3.126700e+08,4769.930176,34520000.0,0.044875,NaN,NaN,6.229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-26,581.409973,3.636300e+08,1038.050049,4.266700e+08,4765.600098,35490000.0,0.044887,NaN,NaN,6.259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-15,6615.279785,5.045020e+09,22348.750000,8.904030e+09,45883.449219,401500000.0,0.013546,0.006052,0.015259,4.034,...,0.083737,10.703297,0.005513,0.030563,-0.162280,-0.121854,-0.190857,20.004532,-0.002007,-0.193813
2025-09-16,6606.759766,5.359510e+09,22333.960938,8.592240e+09,45757.898438,443400000.0,0.013565,0.006057,0.015301,4.026,...,0.073505,10.704463,0.005621,0.026656,-0.013997,-0.210470,-0.051943,20.003707,-0.001604,-0.093724
2025-09-17,6600.350098,5.805340e+09,22261.330078,9.325980e+09,46018.320312,509830000.0,0.013582,0.006069,0.015219,4.076,...,0.086245,10.705832,0.006047,0.030963,-0.128813,0.202287,0.186540,20.006668,-0.002781,0.042920


### Preprocessing Data

Let's check for NaNs

In [67]:
data.isna().sum()

S&P_Close                   0
S&P_Volume                  0
NASDAQ_Close                0
NASDAQ_Volume               0
DJIA_Close                  0
                           ..
DJIA_vlm_1m                21
DJIA_vlm_3m                63
DJIA_3m_rolling_volume     62
DJIA_trend_speed_volume    67
DJIA_trend_dist_volume     62
Length: 61, dtype: int64

Impute some NaNs with average

In [68]:
data['ten_yr'] = data['ten_yr'].fillna(data['ten_yr'].mean())
data['three_m'] = data['three_m'].fillna(data['three_m'].mean())
data['dollar_index'] = data['dollar_index'].fillna(data['dollar_index'].mean())
data['BAMLC0A4CBBB'] = data['BAMLC0A4CBBB'].fillna(data['BAMLC0A4CBBB'].mean())
data['S&P_ret'] = data['S&P_ret'].fillna(data['S&P_ret'].mean())
data['NASDAQ_ret'] = data['NASDAQ_ret'].fillna(data['NASDAQ_ret'].mean())
data['DJIA_ret'] = data['DJIA_ret'].fillna(data['DJIA_ret'].mean())

Drop others

In [69]:
data = data.dropna()
data

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,DJIA_mom_3m,DJIA_3m_rolling_price,DJIA_trend_speed_price,DJIA_trend_dist_price,DJIA_vlm_1w,DJIA_vlm_1m,DJIA_vlm_3m,DJIA_3m_rolling_volume,DJIA_trend_speed_volume,DJIA_trend_dist_volume
Date,,,,,,,,,,,,,,,,,,,,,
2003-12-24,1094.040039,5.180600e+08,1969.229980,6.426300e+08,10305.190430,86180000.0,0.028232,0.000460,0.033565,4.187,...,0.097918,9.189615,0.006395,0.050788,-0.731953,-0.802775,-0.952750,19.109229,-0.020794,-0.837280
2003-12-26,1095.890015,3.560700e+08,1973.140015,5.308100e+08,10324.669922,49520000.0,0.028211,0.000461,0.033542,4.148,...,0.103116,9.191251,0.007069,0.051040,-1.482164,-1.248021,-1.524114,19.085037,-0.042093,-1.367149
2003-12-29,1109.479980,1.058800e+09,2006.479980,1.413210e+09,10450.000000,156400000.0,0.027841,0.000454,0.033091,4.230,...,0.107997,9.192966,0.007592,0.061392,-0.581587,-0.036475,-0.269666,19.080756,-0.052388,-0.212829
2003-12-30,1109.640015,1.012600e+09,2009.880005,1.544270e+09,10425.040039,132800000.0,0.027836,0.000452,0.033202,4.279,...,0.116882,9.194821,0.008232,0.057145,-0.221516,0.517373,-0.562623,19.071826,-0.059447,-0.367471
2003-12-31,1111.920044,1.027500e+09,2003.369995,1.775710e+09,10453.919922,138670000.0,0.027811,0.000453,0.033125,4.257,...,0.098933,9.196391,0.008331,0.058341,-0.085779,-0.495712,-0.714692,19.060481,-0.063871,-0.312874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-12,6584.290039,4.641640e+09,22141.099609,9.023140e+09,45834.218750,421730000.0,0.013618,0.006104,0.015283,4.061,...,0.064584,10.701967,0.005174,0.030819,-0.281641,-0.212210,-0.077601,20.007561,0.003091,-0.147685
2025-09-15,6615.279785,5.045020e+09,22348.750000,8.904030e+09,45883.449219,401500000.0,0.013546,0.006052,0.015259,4.034,...,0.083737,10.703297,0.005513,0.030563,-0.162280,-0.121854,-0.190857,20.004532,-0.002007,-0.193813
2025-09-16,6606.759766,5.359510e+09,22333.960938,8.592240e+09,45757.898438,443400000.0,0.013565,0.006057,0.015301,4.026,...,0.073505,10.704463,0.005621,0.026656,-0.013997,-0.210470,-0.051943,20.003707,-0.001604,-0.093724


### Split data

Training/testing 80/20 split

In [70]:
import math

In [71]:
cutoff = math.floor(len(data)*.8)
training_data = data.iloc[:cutoff]
testing_data = data.iloc[cutoff:]

In [72]:
display(training_data.tail(5))
display(testing_data.head(5))

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,DJIA_mom_3m,DJIA_3m_rolling_price,DJIA_trend_speed_price,DJIA_trend_dist_price,DJIA_vlm_1w,DJIA_vlm_1m,DJIA_vlm_3m,DJIA_3m_rolling_volume,DJIA_trend_speed_volume,DJIA_trend_dist_volume
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-17,3931.330078,4.730650e+09,13965.490234,7.280500e+09,31613.019531,343510000.0,0.019765,0.005560,0.020122,1.301,...,0.069863,10.324463,0.005708,0.036861,0.108435,-0.231522,-0.015311,19.718531,-0.029276,-0.063804
2021-02-18,3913.969971,4.793650e+09,13865.360352,6.499420e+09,31493.339844,325990000.0,0.019849,0.005584,0.020188,1.287,...,0.050232,10.325260,0.005311,0.032271,-0.001410,-0.170006,-0.249308,19.714574,-0.021742,-0.112196
2021-02-19,3906.709961,4.845320e+09,13874.459961,6.737820e+09,31494.320312,353070000.0,0.019884,0.005608,0.021787,1.345,...,0.055858,10.326147,0.005149,0.031416,0.230735,-0.086525,-0.053249,19.713728,-0.012568,-0.031552
2021-02-22,3876.500000,5.917100e+09,13533.049805,6.483740e+09,31521.689453,387210000.0,0.020039,0.005758,0.021767,1.370,...,0.068375,10.327232,0.005158,0.031199,0.386471,-0.078379,0.011324,19.713908,-0.006691,0.060570
2021-02-23,3881.370117,6.296610e+09,13465.200195,7.516510e+09,31537.349609,460410000.0,0.020014,0.005775,0.021754,1.362,...,0.067351,10.328301,0.004947,0.030627,0.335422,0.054383,0.329003,19.719130,0.000356,0.228498


,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,DJIA_mom_3m,DJIA_3m_rolling_price,DJIA_trend_speed_price,DJIA_trend_dist_price,DJIA_vlm_1w,DJIA_vlm_1m,DJIA_vlm_3m,DJIA_3m_rolling_volume,DJIA_trend_speed_volume,DJIA_trend_dist_volume
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-24,3925.429932,6.012790e+09,13597.969727,5.886230e+09,31961.859375,410150000.0,0.019796,0.005728,0.021472,1.389,...,0.088203,10.329701,0.005238,0.042597,0.177307,-0.142340,0.295064,19.723814,0.005283,0.108219
2021-02-25,3829.340088,6.547470e+09,13119.429688,6.390380e+09,31402.009766,454110000.0,0.020285,0.005934,0.021852,1.518,...,0.059393,10.330644,0.005384,0.023983,0.331473,0.196359,0.159398,19.726344,0.011770,0.207506
2021-02-26,3811.149902,6.526070e+09,13192.349609,5.906110e+09,30932.369141,532250000.0,0.020390,0.005910,0.022172,1.460,...,0.029066,10.331105,0.004959,0.008453,0.410447,-0.062311,0.206930,19.729629,0.015900,0.362995
2021-03-01,3901.820068,5.114820e+09,13588.830078,5.079530e+09,31535.509766,385670000.0,0.019907,0.005737,0.021740,1.446,...,0.054177,10.331965,0.004733,0.026904,-0.003985,-0.187356,0.177237,19.732442,0.018534,0.038051
2021-03-02,3870.290039,5.536010e+09,13358.790039,4.948140e+09,31391.519531,337270000.0,0.020064,0.005831,0.021837,1.415,...,0.048333,10.332733,0.004431,0.021561,-0.311234,-0.463847,0.644508,19.742672,0.023542,-0.106278


### Normalize inputs

In [73]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Make sure we only fit on training_data and explanatory variables
targets = ['S&P_next_ret', 'NASDAQ_next_ret', 'DJIA_next_ret']
features = [column for column in training_data.columns if column not in targets]

scaler = StandardScaler()
scaler.fit(training_data[features]) # Fitting on training data

train_scaled = training_data.copy()
test_scaled = testing_data.copy()

train_scaled[features] = scaler.transform(training_data[features])
test_scaled[features] = scaler.transform(testing_data[features])

# Save info on standardization for later
variables_mu = pd.Series(scaler.mean_, index=features)
variables_sd = pd.Series(scaler.scale_, index=features)

### Linear Regression

In [75]:
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, root_mean_squared_error

We are going to test with and without ridge (which will help reduce the impact of collinearity)

In [76]:
# Function to get print results from the models
def eval_and_report(y_true, y_pred, model_name):
    print(f"{model_name:18s} | R^2: {r2_score(y_true, y_pred):.4f} | RMSE: {root_mean_squared_error(y_true, y_pred):.6f}")

In [77]:
# Models
results = {}

for target in targets:
    print(f"\n=== Target: {target} ===")
    X_train = train_scaled[features].copy()
    y_train = train_scaled[target].copy()
    X_test = test_scaled[features].copy()
    y_test = test_scaled[target].copy()

    # 1. Ordinary Least Squares (OLS)
    ols = LinearRegression()
    ols.fit(X_train, y_train)
    yhat_ols = ols.predict(X_test)
    eval_and_report(y_test, yhat_ols, "OLS")

    # Print top coefficients
    ols_coef = pd.Series(ols.coef_, index=features).sort_values(key=np.abs, ascending=False)
    print("Top OLS coeffs:\n", ols_coef.head(10))

    # 2. Ridge with CV over alphas (time-series CV)
    tscv = TimeSeriesSplit(n_splits=5)
    alphas = np.logspace(-4, 3, 30)

    ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
    ridge.fit(X_train, y_train)
    yhat_ridge = ridge.predict(X_test)
    eval_and_report(y_test, yhat_ridge, f"Ridge (alpha={ridge.alpha_:.4g})")

    # Print top coefficients
    ridge_coef = pd.Series(ridge.coef_, index=features).sort_values(key=np.abs, ascending=False)
    print("Top Ridge coeffs:\n", ridge_coef.head(10))

    # Store for later use
    results[target] = {
        "ols_model": ols,
        "ridge_model": ridge,
        "ols_coefs": ols_coef,
        "ridge_coefs": ridge_coef,
        "yhat_ols": pd.Series(yhat_ols, index=y_test.index, name=f"{target}_OLS_pred"),
        "yhat_ridge": pd.Series(yhat_ridge, index=y_test.index, name=f"{target}_Ridge_pred"),
    }


=== Target: S&P_next_ret ===
OLS                | R^2: 0.7641 | RMSE: 0.021697
Top OLS coeffs:
 S&P_mom_3m              -0.030727
S&P_ret                  0.026786
S&P_Close                0.025671
S&P_trend_dist_price     0.023617
S&P_trend_speed_price    0.018151
NASDAQ_mom_3m            0.014361
NASDAQ_Close            -0.014318
S&P_3m_rolling_price    -0.012883
DJIA_mom_3m              0.011899
DJIA_trend_dist_price   -0.011096
dtype: float64
Ridge (alpha=108.3) | R^2: 0.8703 | RMSE: 0.016090
Top Ridge coeffs:
 S&P_ret                  0.018561
DJIA_ret                 0.012236
NASDAQ_ret               0.008810
S&P_trend_dist_price     0.006426
DJIA_mom_3m             -0.002625
S&P_trend_speed_price    0.001764
ten_yr                  -0.001440
GDP                      0.001303
DJIA_3m_rolling_price   -0.001269
S&P_3m_rolling_price    -0.001266
dtype: float64

=== Target: NASDAQ_next_ret ===
OLS                | R^2: 0.7753 | RMSE: 0.028459
Top OLS coeffs:
 NASDAQ_ret             

Plot these results

In [78]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# TODO: What is the blip of data in 2023?

In [81]:
# Define pairs: (pretty name, current-return col in testing_data, next-return target key in results)
pairs = [
    ("S&P 500", "S&P_ret",    "S&P_next_ret"),
    ("NASDAQ",  "NASDAQ_ret", "NASDAQ_next_ret"),
    ("DJIA",    "DJIA_ret",   "DJIA_next_ret"),
]

def get_pred(results_dict, target_key, kind):
    """
    kind: 'ols' or 'ridge'
    Prefer unstandardized preds if present, otherwise fall back to raw.
    """
    unstd_key = f"yhat_{kind}_unstd"
    std_key   = f"yhat_{kind}"
    if target_key in results_dict:
        if unstd_key in results_dict[target_key]:
            return results_dict[target_key][unstd_key]
        elif std_key in results_dict[target_key]:
            return results_dict[target_key][std_key]
    return None  # not found

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.06,
    subplot_titles=[p[0] for p in pairs]
)

for i, (label, cur_col, next_col) in enumerate(pairs, start=1):
    # True = current return from testing_data (unscaled)
    if cur_col not in testing_data.columns:
        continue
    y_true = testing_data[cur_col].dropna().sort_index().rename("True")

    # Predictions target the *next* return (already indexed by the decision time)
    yhat_ols   = get_pred(results, next_col, "ols")
    yhat_ridge = get_pred(results, next_col, "ridge")

    # Align by testing_data's current-return index
    parts = [y_true]
    if yhat_ols is not None:   parts.append(yhat_ols.rename("OLS").reindex(y_true.index))
    if yhat_ridge is not None: parts.append(yhat_ridge.rename("Ridge").reindex(y_true.index))

    df = pd.concat(parts, axis=1).dropna(how="any")
    if df.empty:
        continue

    show_leg = (i == 1)
    fig.add_trace(go.Scatter(x=df.index, y=df["True"],  name="Current return",
                             mode="lines", line=dict(width=1.6), showlegend=show_leg,
                             legendgroup="true"),
                  row=i, col=1)
    if "OLS" in df:
        fig.add_trace(go.Scatter(x=df.index, y=df["OLS"], name="Pred (OLS, next)",
                                 mode="lines", line=dict(width=1.4, dash="dash"),
                                 showlegend=show_leg, legendgroup="ols"),
                      row=i, col=1)
    if "Ridge" in df:
        fig.add_trace(go.Scatter(x=df.index, y=df["Ridge"], name="Pred (Ridge, next)",
                                 mode="lines", line=dict(width=1.4, dash="dot"),
                                 showlegend=show_leg, legendgroup="ridge"),
                      row=i, col=1)

fig.update_layout(
    title="Testing: Current Log Returns vs Next-Period Predictions (OLS & Ridge)",
    height=900,
    hovermode="x unified",
    template="plotly_white",
    margin=dict(t=80, r=30, b=80, l=70),
    legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
)

for r in range(1, 4):
    fig.update_yaxes(title_text="Log return", row=r, col=1)

# Range tools on the bottom axis
fig.update_xaxes(
    rangeselector=dict(buttons=[
        dict(step="all", label="All"),
        dict(count=3, step="year", stepmode="backward", label="3Y"),
        dict(count=1, step="year", stepmode="backward", label="1Y"),
        dict(count=6, step="month", stepmode="backward", label="6M"),
        dict(count=1, step="month", stepmode="backward", label="1M"),
    ]),
    rangeslider=dict(visible=True),
    row=3, col=1
)

fig.show()
